In [ ]:
def log_progress(sequence, every=None, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )

In [ ]:
import pyreadstat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
from scipy.stats import norm
import seaborn as sns
from collections import Counter
import plotly.express as px

In [ ]:
df, meta = pyreadstat.read_sav("r27hall31.sav", apply_value_formats=True, formats_as_category=False, user_missing=False) #импорт для файлов типа sav

df.columns = df.columns +' ' + meta.column_labels

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
#получаем список имен столбиков
colnames = list(df.columns)
binaries = []

In [ ]:
df.info() #техническая ячейка

In [ ]:
#если дропаем определенные значения ячеек (вариант для большого количества пропусков) - активировать ячейку
#df.drop(df.loc[df['Являетесь ли Вы студентом российского вуза?']=='Нет'].index, inplace=True) - на случай, если происходит нечто совершенно специфическое

#и заменяем пропуски в разных столбцах

for i in colnames:
    if(df[i].dtype == np.object):
        df[i].fillna('пропуск', inplace = True)
        
    if(df[i].dtype == np.int32):
        df[i].fillna(df[i].mean(), inplace = True)
        
    if(df[i].dtype == np.float64):
        df[i].fillna(df[i].mean(), inplace = True)

In [ ]:
df1 = df.copy()
df = df.dropna(axis='columns')
len(df)

In [ ]:
print(set(df1.columns) - set(df.columns))

In [ ]:
#получаем новый список имен столбиков (на случай, если была чистка пустых столбиков)
colnames = list(df.columns)
binaries = []

In [ ]:
scale = []
nominal = []
ordinal = []
nominal_with_free_answers = []

partly_ordinal = [] # список для тех ординальных, у которых есть отклонения
free_answers = []
multiple_answers = []

In [ ]:
for i in colnames:
    if(df[i].dtype == np.object):
        df[i] = df[i].str.replace('(', '.')
        df[i] = df[i].str.replace(')', '.')

In [ ]:
#1. сперва создать список с названиями столбиков
#2. путем итераций обработать по отдельности каждый столбик 
#3. по выделенным маркерам добавить каждый столбик в список по типам

#ищем scale
for i in colnames:
    if(df[i].dtype == np.float64 or df[i].dtype == np.int64):
        if len(df[i].value_counts()) > 10:
            scale.append(i)
        else:
            ordinal.append(i)

In [ ]:
a = []

In [ ]:
match = ['хорошо', 
         'нормально', 
         'плохо', 
         'слабо', 
         'средне', 
         'сильно', 
         '18-24', 
         '25-34', 
         '35-44', 
         '45-54', 
         'от 55', 
         'Среднее общее', 
         'Среднее профессиональное', 
         'Неоконченное высшее', 
         'Высшее', 
         'Уровень дохода крайне низкий,денег не хватает даже на питание', 
         'На питание денег хватает, но с покупкой одежды бывают трудности', 
         'Покупка продуктов питания и одежды не вызывает трудностей, но для приобретения предметов длительного пользования .холодильник, телевизор. приходится брать взаймы', 
         'В целом, материальных проблем нет, но покупка действительно дорогих вещей .машина, квартира. вызывает затруднение', 
         'Нет материальных проблем, могу легко позволить себе недвижимость или машину',
         'студент специалитета',
         'студент бакалавриата',
         'студент магистратуры',
         'студент аспирантуры',
         'Нет, это невозможно',
         'Скорее невозможно',
         'Затрудняюсь ответить',
         'g',
         'Скорее возможно',
         'Да, это возможно',
         'Важно',
         'Скорее важно',
         'g',
         'Затрудняюсь ответить',
         'Скорее не важно',
         'Совершенно не важно',
         'Да, слышал и принимал участие',
         'Слышал, но не принимал участия',
         'g',
         'Нет, не слышал.',
         'Да, считаю это важным',
         'Не имею четкой позиции, мне все равно',
         'g',
         'Нет, не поддерживаю']
#в список нельзя добавлять элементы со скобками
#на основе списков строится словарь для распознавания Ordinal шкалы

In [ ]:
rep = [3,2,1, 1,2,3, 1,2,3,4,5, 1,2,3,4, 1,2,3,4,5, 1,2,3,4, 1,2,3,3,4,5, 5,4,3,3,2,1, 3,2,1,1, 3,2,2,1]
len(rep)

In [ ]:
change_dict = dict(zip(match, rep))

In [ ]:
change_dict

In [ ]:
def string_finder(row, words):
    if any(word in field for field in row for word in words):
        return True
    return False


In [ ]:
#деревья решений

n = 0
for i in colnames:
    if(df[i].dtype == np.object):
        a.append(i)
        n+=1
    
        #определяем, содержит ли столбик значения из словаря Ordinal в достаточном количестве
        for i1 in a: #i1 - это названия колонок по порядку
            wrkdf = df[i1].str.contains('|'.join(match)).value_counts()
            try:
                if wrkdf[0] < wrkdf[1]*5: #Если труъ больше, чем фолс / возможно, надо будет додумать какие-то пропорции, но пока так
                    partly_ordinal.append(i1)
                else:
                    if len(df[i1].value_counts()) > 10: #более десяти оригинальных текстовых ответов
                        count = df[i1].value_counts()
                        count0 = pd.DataFrame(count).T.columns.value_counts()

                        datframe = pd.DataFrame(count0)
                        datframe['index'] = datframe.index
                        wrkdf = datframe['index'].index.str.contains('|'.join(','))
                        wrkdf = pd.DataFrame(wrkdf)[0].value_counts()

                        try:
                            if wrkdf[0] < wrkdf[1]: #Если труъ больше, чем фолс, запятых больше
                                multiple_answers.append(i1)
                            else:    #определяем, свободные ответы или номинальная с ф-цией сврбодных ответов
                                if np.std(df[i1].value_counts()) > 8:
                                    nominal_with_free_answers.append(i1)
                                else:
                                    free_answers.append(i1) 
                        except:
                            pass
    
                        if len(wrkdf) == 1 and list(pd.DataFrame(wrkdf).T.columns)[0] == True: #все с запятыми
                            multiple_answers.append(i1)
                
                        if len(wrkdf) == 1 and list(pd.DataFrame(wrkdf).T.columns)[0] == False: #все без запятых
                            if np.std(df[i1].value_counts()) > 8:
                                nominal_with_free_answers.append(i1)
                            if np.std(df[i1].value_counts()) < 8 or np.std(df[i1].value_counts()) == 8:
                                free_answers.append(i1) 
            except:
                pass
            
            if len(wrkdf) == 1 and list(pd.DataFrame(wrkdf).T.columns)[0] == True: #В листе только совпадения и он из одной ячейки
                ordinal.append(i1) 
    
            if len(wrkdf) == 1 and list(pd.DataFrame(wrkdf).T.columns)[0] == False: #В листе вообще нет совпадений по словарю Ordinal
                if len(df[i1].value_counts()) > 10: #более десяти оригинальных текстовых ответов
                    count = df[i1].value_counts()
                    count0 = pd.DataFrame(count).T.columns.value_counts()

                    datframe = pd.DataFrame(count0)
                    datframe['index'] = datframe.index
                    wrkdf = datframe['index'].index.str.contains('|'.join(','))
                    wrkdf = pd.DataFrame(wrkdf)[0].value_counts()

                    try:
                        if wrkdf[0] < wrkdf[1]: #Если труъ больше, чем фолс, запятых больше
                            multiple_answers.append(i1)
                        else:    #определяем, свободные ответы или номинальная с ф-цией сврбодных ответов
                            if np.std(df[i1].value_counts()) > 8:
                                nominal_with_free_answers.append(i1)
                            else:
                                free_answers.append(i1) 
                    except:
                        pass
    
                    if len(wrkdf) == 1 and list(pd.DataFrame(wrkdf).T.columns)[0] == True: #все с запятыми
                        multiple_answers.append(i1)
                
                    if len(wrkdf) == 1 and list(pd.DataFrame(wrkdf).T.columns)[0] == False: #все без запятых
                        if np.std(df[i1].value_counts()) > 8:
                                nominal_with_free_answers.append(i1)
                        if np.std(df[i1].value_counts()) < 8 or np.std(df[i1].value_counts()) == 8:
                            free_answers.append(i1) 
                    
                    
                            
                else:
                    nominal.append(i1) #если меньше десяти, то уходит в номинальную

            
        

In [ ]:
nominal = set(nominal)
scale = set(scale)

ordinal = set(ordinal)
nominal_with_free_answers = set(nominal_with_free_answers)

partly_ordinal = set(partly_ordinal) # список для тех ординальных, у которых есть отклонения
free_answers = set(free_answers)
multiple_answers = set(multiple_answers)

In [ ]:
print(len(colnames))
print(len(nominal) + len(scale)+len(ordinal)+len(nominal_with_free_answers)+len(partly_ordinal)+len(free_answers)+len(multiple_answers))

#проверка, не упустили ли мы при использовании десижн триз какие-то шкалы

In [ ]:
check = []
types = [list(nominal), list(scale), list(ordinal), list(nominal_with_free_answers), list(partly_ordinal), list(free_answers), list(multiple_answers)]

In [ ]:
for i in types:
    for i in i:
        check.append(i)


In [ ]:
result=list(set(colnames) - set(check))
result
#тут мы узнаём, какие именно ячейки выпали при десижн триз. Если это метки времени, то всё нормально

In [ ]:
#тут мы смотрим, какие столбцы в какие категории попали

multiple_answers

In [ ]:
nominal

In [ ]:
nominal_with_free_answers

In [ ]:
free_answers

In [ ]:
ordinal

In [ ]:
scale

In [ ]:
for i in ordinal: #автоматическая перекодировка по словарю
    if(df[i].dtype == np.object):
        df[i].replace(change_dict,inplace=True)
#надо будет добавить лог замен

In [ ]:


for i in nominal_with_free_answers:
    ff = []
    z = 0
    
    for x in df[i].value_counts(): #определяем номинальные индексы без "другого"
            
        if x/len(df[i]) > 0.05:
            ff.append(df[i].value_counts().index.tolist()[z]) #фильтр для датафрейма
        z +=1 
   
    
    df_filter = df[i].isin(ff) 
    df[i+' (закрытые ответы)'] = df[df_filter][i]
    df[i+' (закрытые ответы)'].fillna(('другое'), inplace=True)
    df[i+' (открытые ответы)'] = df[~df_filter][i]
    
    #добавляем закрытые ответы в nominal, открытые - во free_answers
    
    a = str(i+' (закрытые ответы)')
    b = str(i+' (открытые ответы)')
    
    #добавляем в релевантные группы
    nominal.add(a)
    free_answers.add(b)

In [ ]:
free_answers

In [ ]:
multiple_answers

In [ ]:
for i in multiple_answers:
    newlist = []
    mylist = list(df[i])
    for element in mylist:
       newlist.extend(element.split(',')) #множим элемементы через запятую

    newlist = [x.strip(' ') for x in newlist] #удаляем пробелы в начале и конце
    
    dfdf = pd.DataFrame(newlist)
    dfdf[0].value_counts()
    
    ff = []
    z = 0
    for x in dfdf[0].value_counts(): #частотный анализ в строках
        if x/len(dfdf[0]) > 0.05:
            ff.append(dfdf[0].value_counts().index.tolist()[z])
        z+=1
    
    df['открытые ответы _ ' + i] = df[i]
    
    for e in ff:
        df[e + ' _ ' + i] = df[i].str.contains('|'.join(e)) #создаем новые столбцы
        binaries.append(e + ' _ ' + i)
        df[e + ' _ ' + i] *= 1 #меняем тру/фолс на 1/0
        
        df['открытые ответы _ ' + i] = df['открытые ответы _ ' + i].str.replace(',', ' ')
        df['открытые ответы _ ' + i] = df['открытые ответы _ ' + i].str.replace(e, '')
        
        free_answers.add('открытые ответы _ ' + i)

# Второй этап

binaries - лист с даммис по бинарным ответам

types - список всех остальных форматов
(types = [list(nominal), list(scale), list(ordinal), list(nominal_with_free_answers), list(partly_ordinal), list(free_answers), list(multiple_answers)])

их миксуем:
scale 
nominal 
ordinal 
+ binaries - частный пример ordinai (соединил с ординал)

free_answers - по ним просто проводим семантический частотный анализ, не участвуют в общей тасовке

nominal_with_free_answers - переформировано в закрытые ответы (в номинал) и в открытые ответы (в фри энсерс)
partly_ordinal - список для тех ординальных, у которых есть отклонения. Отправляется в переработку
multiple_answers - заменены на binaries и free answers


In [ ]:
mix_list = ('scale', 'nominal', 'ordinal')

In [ ]:
ordinal = ordinal.union(set(binaries)) #объединим бинарис с ординал

In [ ]:
import itertools

c = list(itertools.product(mix_list, repeat=2))

In [ ]:
c

пирсон:
('scale', 'scale'),

спирман:
 ('scale', 'ordinal'),
 ('ordinal', 'ordinal')

хи-квадрат:
 ('nominal', 'scale'),
 ('nominal', 'nominal'),
 ('nominal', 'ordinal'),


In [ ]:
pearson = list(itertools.combinations(scale, 2))
pearson = set(pearson)

In [ ]:
spearman = list(itertools.combinations(scale.union(ordinal), 2))
spearman.extend(list(itertools.combinations(ordinal, 2)))
spearman = set(spearman)

In [ ]:
chi_square = list(itertools.combinations(nominal.union(scale), 2))
chi_square.extend(list(itertools.combinations(nominal, 2)))
chi_square.extend(list(itertools.combinations(nominal.union(ordinal), 2))) #вариант без пар, внутри которых оба элемента одинаковые
chi_square = set(chi_square)

In [ ]:
chi_square = chi_square - spearman
spearman = spearman - pearson

In [ ]:
from scipy.stats.stats import pearsonr

pearson_fin = []

for i in pearson:
    pearson_fin.append(pearsonr(df[i[0]], df[i[1]]))

In [ ]:
spear_fin = []

from scipy import stats

for i in spearman:
    spear_fin.append(stats.spearmanr(df[i[0]], df[i[1]]))


In [ ]:
import scipy

chi_square_fin = []

for i in chi_square:
    table = pd.crosstab(df[i[0]], df[i[1]])
    chi2, prob, daf, expected = scipy.stats.chi2_contingency(table)
    chi_square_fin.append(float('{:.6f}'.format(prob))) #ограничитель на 6 цифр после запятой
    

In [ ]:
chi_square_fin_frame = pd.DataFrame(chi_square_fin)

if len(chi_square_fin) !=0:
    chi_square_fin_frame = chi_square_fin_frame.astype(np.float16)
    chi_square_fin_frame['pair'] = chi_square
    chi_square_fin_frame.rename(columns={0: 'pvalue'}, inplace=True)
    chi_square_fin_frame = chi_square_fin_frame.sort_values('pvalue')
    chi_square_fin_frame = chi_square_fin_frame.reset_index()
    del chi_square_fin_frame['index']

    qq = chi_square_fin_frame

    qq['pvalue level'] = qq['pvalue']
    qq.loc[qq['pvalue'] < 0.5, 'pvalue level'] = 'значимо'
    qq.loc[qq['pvalue'] < 0.3, 'pvalue level'] = 'крайне значимо'
    qq.loc[qq['pvalue'] > 0.5, 'pvalue level'] = 'не значимо'
    qq.loc[qq['pvalue'] == 0.5, 'pvalue level'] = 'не значимо'
    
else:
    print('нет корреляций по хи-квадрату')
    
chi_square_fin_frame[:200]

In [ ]:
if len(chi_square_fin) !=0:
    writer = pd.ExcelWriter('chi_square.xlsx', engine='xlsxwriter')

    chi_square_fin_frame.to_excel(writer,
                    sheet_name='Sheet_name_1')

    writer.save()
else:
    print('нет корреляций по хи-квадрату')

In [ ]:
spear_frame = pd.DataFrame(spear_fin)

if len(spear_fin) !=0:
    spear_frame = spear_frame.astype(np.float16)

    spear_frame['pair'] = spearman

    spear_frame = spear_frame.sort_values('correlation', ascending=False)
    spear_frame = spear_frame.reset_index()
    del spear_frame['index']


    spear_frame = spear_frame.dropna()

    qq = spear_frame
    qq['corr level'] = qq['correlation']
    qq.loc[qq['correlation'] == 0, 'corr level'] = 'отсутствует'
    qq.loc[qq['correlation'] > 0, 'corr level'] = 'слабая прямая'
    qq.loc[qq['correlation'] > 0.34, 'corr level'] = 'умеренная прямая'
    qq.loc[qq['correlation'] > 0.67, 'corr level'] = 'сильная прямая'

    qq.loc[qq['correlation'] < 0, 'corr level'] = 'слабая обратная'
    qq.loc[qq['correlation'] < -0.34, 'corr level'] = 'умеренная обратная'
    qq.loc[qq['correlation'] < -0.67, 'corr level'] = 'сильная обратная'

    qq['pvalue level'] = qq['pvalue']
    qq.loc[qq['pvalue'] < 0.5, 'pvalue level'] = 'значимо'
    qq.loc[qq['pvalue'] < 0.3, 'pvalue level'] = 'крайне значимо'
    qq.loc[qq['pvalue'] > 0.5, 'pvalue level'] = 'не значимо'
    qq.loc[qq['pvalue'] == 0.5, 'pvalue level'] = 'не значимо'

    
else:
    print('нет корреляций по спирмену')
    
spear_frame[:1000]

In [ ]:
if len(spear_fin) !=0:
    writer = pd.ExcelWriter('spearman.xlsx', engine='xlsxwriter')

    spear_frame.to_excel(writer,
                    sheet_name='Sheet_name_1')

    writer.save()
else:
    print('нет корреляций по спирмену')

In [ ]:
pearson_frame = pd.DataFrame(pearson_fin)

if len(pearson_fin) !=0:
    pearson_frame = pearson_frame.astype(np.float16)
    pearson_frame['pair'] = pearson
    pearson_frame.rename(columns={0: 'correlation', 1: 'pvalue'}, inplace=True)

    pearson_frame = pearson_frame.sort_values('correlation', ascending=False)
    pearson_frame = pearson_frame.reset_index()
    del pearson_frame['index']

    pearson_frame = pearson_frame.dropna()

    qq = pearson_frame
    qq['corr level'] = qq['correlation']
    qq.loc[qq['correlation'] == 0, 'corr level'] = 'отсутствует'
    qq.loc[qq['correlation'] > 0, 'corr level'] = 'слабая прямая'
    qq.loc[qq['correlation'] > 0.34, 'corr level'] = 'умеренная прямая'
    qq.loc[qq['correlation'] > 0.67, 'corr level'] = 'сильная прямая'

    qq.loc[qq['correlation'] < 0, 'corr level'] = 'слабая обратная'
    qq.loc[qq['correlation'] < -0.34, 'corr level'] = 'умеренная обратная'
    qq.loc[qq['correlation'] < -0.67, 'corr level'] = 'сильная обратная'

    qq['pvalue level'] = qq['pvalue']
    qq.loc[qq['pvalue'] < 0.5, 'pvalue level'] = 'значимо'
    qq.loc[qq['pvalue'] < 0.3, 'pvalue level'] = 'крайне значимо'
    qq.loc[qq['pvalue'] > 0.5, 'pvalue level'] = 'не значимо'
    qq.loc[qq['pvalue'] == 0.5, 'pvalue level'] = 'не значимо'


    
else:
    print('нет корреляций по пирсону')
    
pearson_frame[:1000]

In [ ]:
if len(pearson_fin) !=0:
    writer = pd.ExcelWriter('pearson.xlsx', engine='xlsxwriter')

    pearson_frame.to_excel(writer,
                    sheet_name='Sheet_name_1')

    writer.save()
    
else:
    print('нет корреляций по пирсону')

# ВИЗУАЛИЗАЦИЯ

In [ ]:
for i in ordinal:
    z = df[i].value_counts()
    z = pd.DataFrame(z)
    z['name'] = z.index
    z = z.sort_values(by=['name'])
    z = z.applymap(str)


    fig = px.histogram(z, x='name', y=z.columns[0], histfunc='avg', nbins=len(z))
    fig.show()

In [ ]:
for i in ordinal:
    sns.countplot(df[i], color = "black")
    plt.show();

In [ ]:
for i in nominal:
    z = df[i].value_counts()
    z = pd.DataFrame(z)
    z['name'] = z.index
    z = z.sort_values(by=['name'])
    z = z.applymap(str)


    fig = px.histogram(z, x='name', y=z.columns[0], histfunc='avg', nbins=len(z))
    fig.show()

In [ ]:
for i in nominal:
    sns.countplot(df[i], color = "black")
    plt.show();

In [ ]:
for i in binaries:
    print('процент положительных ответов:', i, df[i].value_counts()[1]/len(df[i]),'\n')

In [ ]:
for i in free_answers:
    print(i,':\n', Counter(df[i].to_string(index=False).replace('\n','').lower().split()).most_common(40),'\n')

In [ ]:
#чтобы как в спсс

import statsmodels.api as sm

X_train2 = np.c_[X_train, np.ones_like(X_train)]

X_train2 = np.c_[X_train, np.ones_like(X_train)]
ols = sm.OLS(y_train, exog=X_train2)
model = ols.fit()
model.summary2() 